![Image text](картинка2.png)



## 1. Детальное построение решётки ролей

### Исходные роли и их права

Давайте конкретизируем права каждой роли:

- **User**: 
  - Чтение публичного контента
  - Просмотр своего профиля
  - Не может изменять данные

- **Editor**: 
  - Все права User +
  - Создание, редактирование, удаление контента
  - Модерация комментариев

- **Admin**:
  - Все права Editor +
  - Управление пользователями (блокировка, назначение ролей)
  - Настройка системы
  - Просмотр базовой статистики

- **SuperAdmin**:
  - Все права Admin +
  - Управление другими администраторами
  - Полный доступ к системным настройкам
  - Экспорт данных

### Математическое представление

Решётка - это частично упорядоченное множество, где для любых двух элементов существует:
- **Точная верхняя грань** (least upper bound) - наименьший элемент, который ≥ обоим
- **Точная нижняя грань** (greatest lower bound) - наибольший элемент, который ≤ обоим

Для наших ролей:
```
SuperAdmin
    |
   Admin
    |
  Editor
    |
   User
```

**Проверим свойства решётки:**

Возьмём пару User и Editor:
- Верхняя грань: Editor (минимальная роль, которая имеет права обоих)
- Нижняя грань: User (максимальная роль, права которой есть у обоих)

Возьмём пару Editor и Admin:
- Верхняя грань: Admin
- Нижняя грань: Editor

✅ Все пары имеют верхние и нижние грани - это действительно решётка.

## 2. Глубокий анализ модулярности

### Что такое модулярность?

Модулярная решётка удовлетворяет условию:
Если \( a ≤ c \), то \( a ∨ (b ∧ c) = (a ∨ b) ∧ c \)

Где $\lor$ — операция объединения (наибольшая верхняя грань, т.е. роль с наибольшими правами), а $\land$ — операция пересечения (наибольшая нижняя грань, т.е. роль с наименьшими правами).

**Простыми словами:** "Если a меньше c, то можно без опасения комбинировать a с пересечением b и c"

### Проверим на конкретных примерах

**Пример 1:** \( a = User, b = Admin, c = Editor \)
- Условие: User ≤ Editor ✓
- Левая часть: User ∨ (Admin ∧ Editor) = User ∨ Editor = Editor
- Правая часть: (User ∨ Admin) ∧ Editor = Admin ∧ Editor = Editor
- Равенство: Editor = Editor ✓

**Пример 2:** \( a = Editor, b = SuperAdmin, c = Admin \)
- Условие: Editor ≤ Admin ✓
- Левая часть: Editor ∨ (SuperAdmin ∧ Admin) = Editor ∨ Admin = Admin
- Правая часть: (Editor ∨ SuperAdmin) ∧ Admin = SuperAdmin ∧ Admin = Admin
- Равенство: Admin = Admin ✓

**Почему это важно?** Модулярность гарантирует, что наша система прав будет вести себя предсказуемо при комбинации ролей и наследовании прав.

## 3. Добавление Auditor: полный анализ

### Права Auditor:
- Чтение логов аудита
- Чтение системных журналов
- Экспорт отчетов
- **НЕТ** прав на изменение данных
- **НЕТ** прав управления пользователями

### Как Auditor вписывается в иерархию?

```
    SuperAdmin
       |
      Admin
       |    \
       |     \
     Editor  Auditor
        \     /
         \   /
          User
```

**Обоснование структуры:**

1. **Auditor > User**: Auditor имеет как минимум права на чтение
2. **Auditor и Editor не сравнимы**:
   - Auditor может читать логи (чего не может Editor)
   - Editor может изменять контент (чего не может Auditor)
3. **Admin > Auditor**: Admin имеет все права Auditor + больше
4. **SuperAdmin > Admin** (и соответственно > Auditor)

### Математические свойства новой решётки

**Проверим, осталась ли решётка модулярной после добавления Auditor**

Возьмём \( a = User, b = Editor, c = Auditor \)
- Условие: User ≤ Auditor ✓
- Левая часть: User ∨ (Editor ∧ Auditor) = User ∨ User = User
- Правая часть: (User ∨ Editor) ∧ Auditor = Editor ∧ Auditor = User
- Равенство: User = User ✓

Решётка осталась модулярной! Это хорошая новость для стабильности системы.


## 4. Объясните, как можно использовать эту решётку для автоматической проверки прав в middleware веб-приложения.

**Как использовать решётку для проверки прав в middleware:**

Middleware — это проверка перед доступом к функции. Решётка даёт чёткое правило: **роль пользователя должна быть ≥ требуемой роли для действия**.

**Алгоритм:**

1. **Храним решётку** как набор отношений между ролями (например, в виде словаря или графа).

2. **Для каждого эндпоинта** указываем минимальную требуемую роль:
   - `GET /posts` → `User`
   - `PUT /posts` → `Editor`  
   - `GET /logs` → `Auditor`
   - `DELETE /users` → `Admin`

3. **При запросе:**
   - Извлекаем роль пользователя из сессии или токена.
   - Сравниваем её с требуемой ролью для запрашиваемого пути.
   - Если роль пользователя **равна или выше** в решётке — пропускаем запрос.
   - Если ниже — возвращаем ошибку 403.

**Критерий "равно или выше" определяется позицией в решётке:**
- `Admin` ≥ `Editor` (админ может всё, что может редактор)
- `Admin` ≥ `Auditor` 
- `Editor` и `Auditor` — несравнимы (нельзя сказать, что один выше другого)

**Пример:**
- Запрос к `/logs` требует `Auditor`.
- Пользователь с ролью `Editor` не получит доступ (так как `Editor` не ≥ `Auditor`).
- Пользователь с ролью `Admin` получит доступ (так как `Admin` ≥ `Auditor`).

**Итог:** Решётка превращает проверку прав в простое сравнение ролей по их положению в иерархии.

In [18]:
from enum import IntEnum

# 1. Определение ролей с использованием IntEnum для задания порядка и весов
# Чем выше значение, тем больше прав у роли.
class Role(IntEnum):
    USER = 1
    EDITOR = 2
    ADMIN = 3
    SUPER_ADMIN = 4

# 2. Функция для проверки отношения наследования прав
def has_permission(user_role: Role, required_role: Role) -> bool:
    """
    Проверяет, имеет ли пользовательская роль достаточные права 
    для выполнения действия, требующего определенной роли.
    """
    # Сравниваем числовые значения ролей
    return user_role.value >= required_role.value

# 3. Примеры использования и демонстрация иерархии (решетки)

print(f"Иерархия ролей (снизу вверх):")
print(f"  {Role.SUPER_ADMIN.name}")
print(f"  {Role.ADMIN.name}")
print(f"  {Role.EDITOR.name}")
print(f"  {Role.USER.name}")

print("\nДемонстрация проверок доступа:")

# Пользователь (User) пытается получить доступ к ресурсу редактора (Editor)
user_role = Role.USER
required_role_editor = Role.EDITOR
print(f"Может ли {user_role.name} получить доступ к {required_role_editor.name}? -> {has_permission(user_role, required_role_editor)}")
# Ожидаемый результат: False

# Администратор (Admin) пытается получить доступ к ресурсу пользователя (User)
admin_role = Role.ADMIN
required_role_user = Role.USER
print(f"Может ли {admin_role.name} получить доступ к {required_role_user.name}? -> {has_permission(admin_role, required_role_user)}")
# Ожидаемый результат: True (Админ имеет все права User)

# SuperAdmin пытается получить доступ к ресурсу Admin
super_admin_role = Role.SUPER_ADMIN
required_role_admin = Role.ADMIN
print(f"Может ли {super_admin_role.name} получить доступ к {required_role_admin.name}? -> {has_permission(super_admin_role, required_role_admin)}")
# Ожидаемый результат: True


Иерархия ролей (снизу вверх):
  SUPER_ADMIN
  ADMIN
  EDITOR
  USER

Демонстрация проверок доступа:
Может ли USER получить доступ к EDITOR? -> False
Может ли ADMIN получить доступ к USER? -> True
Может ли SUPER_ADMIN получить доступ к ADMIN? -> True


Что делает этот код:

IntEnum: Мы используем перечисление IntEnum для присвоения каждой роли уникального числового значения. Это естественным образом определяет отношение частичного порядка: USER (1) < EDITOR (2) < ADMIN (3) < SUPER_ADMIN (4).

has_permission: Эта функция реализует логику решетки: если ваша роль имеет значение больше или равное требуемому, у вас есть разрешение.

Визуализация: Код распечатывает иерархию и демонстрирует проверки доступа, показывая, как работает наследование прав.